# Data Exploration - Milan

This notebook explores demographic and real estate data for Paris to analyze gentrification patterns.

In [85]:
# INITIALIZATION

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd


# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

## Load Data

Load the raw data for Milan from the data directory.

In [98]:
# --- GeoJSON Import ---
nil = gpd.read_file("../data/raw/milan/nil_milano.geojson").set_index("ID_NIL").sort_index()

# --- CSV Import ---
raw_dati_quartieri = pd.read_csv("../data/raw/milan/dati_quartieri_11-21.csv", sep=';')
raw_dati_residenti = pd.read_csv("../data/raw/milan/residenti_per_cittadinanza_e_quartiere.csv", sep=';')
#titoli_studio11 = pd.read_csv("../data/raw/milan/titoli_studio_2011.csv", sep=';', encoding='latin-1')

raw_cens_2021 = pd.read_excel("../data/raw/milan/Censimenti istat/Comuni_2021/Milano_indicatori_2021_sezioni.xlsx")
geo_sez_censimento = gpd.read_file('../data/raw/milan/sez_censimento_istat_geojson/sezioni_censimento_2021.geojson')

irpef_2023 = pd.read_csv('../data/raw/milan/irped2023_mef.csv', sep = ";")
raw_cap_zones = gpd.read_file('../data/raw/milan/cap_italia.geojson')

raw_omi_quot = pd.read_csv('../data/raw/milan/quotazioni_omi_riepilogo.csv', sep = ";")
omi_zones = gpd.read_file('../data/raw/milan/zone_omi24_comunemilano.geojson')

df = pd.DataFrame()

In [40]:
filtered_columns = [
    'Anno',
    'Quartiere',
    'NIL',
    'Totale',
    'Stranieri',
    'Famiglie registrate in anagrafe',
    'Famiglie unipersonali registrate in anagrafe',
    '80 e più soli fam registrate in anagrafe',
    'Nati vivi',
    '80 e più'
]

dati_quartieri = raw_dati_quartieri[filtered_columns]

dati_quartieri.head()

,Anno,Quartiere,NIL,Totale,Stranieri,Famiglie registrate in anagrafe,Famiglie unipersonali registrate in anagrafe,80 e più soli fam registrate in anagrafe,Nati vivi,80 e più
0,2011,Adriano,17.0,14230.0,2538.0,6942.0,2947.0,243.0,160.0,476.0
1,2011,Affori,80.0,23061.0,6276.0,11974.0,5987.0,634.0,229.0,1226.0
2,2011,Assiano,87.0,294.0,27.0,121.0,52.0,6.0,4.0,8.0
3,2011,Baggio - Q.re degli Olmi - Q.re Valsesia,55.0,29594.0,4545.0,14459.0,6033.0,914.0,230.0,2003.0
4,2011,Bande Nere,52.0,43455.0,5915.0,23861.0,11846.0,2040.0,347.0,4443.0


In [41]:
# Merge Male and females
dati_residenti = raw_dati_residenti.groupby(["Anno", "IdNil", "Nil", "Eta", "Cittadinanza"])[["Residenti"]].sum().reset_index()
dati_residenti.head()

,Anno,IdNil,Nil,Eta,Cittadinanza,Residenti
0,1999,1,Duomo,0,"Cinese, Rep. Popolare",2
1,1999,1,Duomo,0,Egitto,1
2,1999,1,Duomo,0,Etiopia,1
3,1999,1,Duomo,0,Filippine,4
4,1999,1,Duomo,0,Italia,175


In [42]:
names = {
    "P1" : "Tot Residents", 
    "P90" : "Res Higher Degree", 
    "P101" : "Res with a job"
}
cens_2021 = raw_cens_2021[["SEZ21_ID", "COM_ASC1", "COM_ASC2", "P1", "P90", "P101"]].rename(columns = names)

cens_2021.head()

,SEZ21_ID,COM_ASC1,COM_ASC2,Tot Residents,Res Higher Degree,Res with a job
0,151460000002,15146001,15146001,15,2,9
1,151460000003,15146001,15146001,18,13,4
2,151460000004,15146001,15146001,13,6,6
3,151460000006,15146001,15146001,4,0,2
4,151460000007,15146001,15146001,4,0,4


In [92]:
cap_zones = raw_cap_zones[raw_cap_zones["comune"] == "Milano"][["cap", "geometry"]].set_index("cap").sort_index()


irpef_s = (irpef_2023[irpef_2023["Denominazione Comune"] == "MILANO"].reset_index())[["Anno di imposta", "CAP", "Reddito complessivo - Frequenza", "Reddito complessivo - Ammontare in euro"]]
irpef_s.head()

,Anno di imposta,CAP,Reddito complessivo - Frequenza,Reddito complessivo - Ammontare in euro
0,2023,20121,20699,1856884494
1,2023,20122,27941,2103106002
2,2023,20123,22118,1896725228
3,2023,20124,24671,1496783232
4,2023,20125,30119,1085896307


In [105]:
omi_zones.head()

omi_quot = raw_omi_quot[["Anno", "Periodo", "Zona", "LinkZona", "Cod_Tip", "Descr_Tipologia", "Compr_min", "Compr_max", "Loc_min", "Loc_max"]]
omi_quot.head()

,Anno,Periodo,Zona,LinkZona,Cod_Tip,Descr_Tipologia,Compr_min,Compr_max,Loc_min,Loc_max
0,2004,2 semestre,B01,MI00000290,20,Abitazioni civili,3850,5450,"10,2","14,5"
1,2004,2 semestre,B01,MI00000290,20,Abitazioni civili,3300,4750,"8,7","12,7"
2,2004,2 semestre,B01,MI00000290,20,Abitazioni civili,5000,6750,"13,2",18
3,2004,2 semestre,B01,MI00000290,21,Abitazioni di tipo economico,4800,6400,16,"21,2"
4,2004,2 semestre,B01,MI00000290,21,Abitazioni di tipo economico,3600,5350,12,"17,7"


## Exploratory Data Analysis

Analyze the characteristics of the neighborhoods in Milan.

In [ ]:
# TODO: Add exploratory analysis